<a href="https://colab.research.google.com/github/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning/blob/main/GB886_II_8_LasVegasExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's load some libraries:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

# Las Vegas Dataset

Let's load the dataset from the course repository.

In [ ]:
!git clone https://github.com/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning.git

In [ ]:
lasvegas = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_II_7_LasVegasTripAdvisorReviews.csv')

And let's take a look:

In [ ]:
lasvegas.head()

In [ ]:
lasvegas.describe()

In [ ]:
lasvegas['User country'].value_counts()

In [ ]:
lasvegas['Hotel name'].value_counts()

## Data Preparation

The first issue we encounter is that there are categorical variables (Period of stay, Traveler type, etc.), continuous/numerical variables (Nr. reviews, Helpful votes), and some where it isn't clear. For instance, Hotel stars could be continuous or ordinal---and really Score as well.


We will treat our dependent variable (Score) as continuous since we are running a linear regression (we will discuss alternatives later). We will treat Stars as categorical and we will drop the 'User country' and 'Hotel name'.

In [ ]:
numerics = list(lasvegas.select_dtypes(include=['int64']).columns)
numerics.remove('Hotel stars')
numerics.remove('Score')
factors = list(lasvegas.select_dtypes(include=['object']).columns)
factors.append('Hotel stars')
factors.remove('User country')
factors.remove('Hotel name')

Let's look at the numerical columns:

In [ ]:
lasvegas_numcols = lasvegas[numerics]
lasvegas_numcols.head()

One aspect that is maybe problematic is that `Helpful votes' is correlated to Nr. reviews---the more reviews there are, the more can be helpful. So we will **engineer** a new feature which considers the proportion of helpful votes divided by the Number of reviews:

In [ ]:
lasvegas_numcols['helpful_proportion'] = lasvegas_numcols['Helpful votes'] / lasvegas_numcols['Nr. reviews']

In [ ]:
lasvegas_numcols.head()

Now we take the categorical data and transfer them into dummies:

In [16]:
lasvegas_faccols = lasvegas[factors]
dummies = pd.get_dummies(lasvegas_faccols, drop_first=True)

And finally we combine the numerical and the categorical columns---plus our outcome varoable---together:

In [ ]:
lasvegas_new = pd.concat([lasvegas_numcols, dummies], axis = 1)
lasvegas_new = pd.concat([lasvegas_new, lasvegas['Score']], axis =1)
lasvegas_new.head()

## Run our Linear Regression

Let's run our linear regression:

In [ ]:
y = lasvegas_new['Score']
X = lasvegas_new.drop(columns=['Score'])
X = sm.add_constant(X)
model_sm = sm.OLS(y, X.astype(float)).fit() #Because of the way data was stored in the df, sm does not work. Have to coerce into numbers.
model_sm.summary()

The regression table provides insights on how features are associated with scores. For instance, hotel starts are positively associated with the predicted score, and so is having a pool.

However, we should be mindful not to attach "causal" interpretations. For instance, even though 'Spa' has a negative association with Score, that likely doesn't mean that closing your Spa and leaving everything else unchanged will positively affect scores. Possibly the mechanism is that having a Spa leads to higher prices and custumers don't like paying higher prices---so closing the spa while charging the same price may not have an effect. Again, we explore when and how to obtain causal inference in more detail in another class in your program (GB 740).

However, in the spirit of this class, we can use the model for generating a prediction---in the spirit of this class!

## Prediction

For generating predictions, we can now take features of a traveler and their profile (how many ratings have they done, when are they traveling, are they going with their family, etc.) and of the hotel (how many stars, does it have free internet, etc.) to predict a satisfaction score for their trip. This may be helpful in recommending a hotel in a given price range, say.

For instance, a traveler that has written 4 total reviews, 2 on hotels, and receeived 2 helpful votes traveling in the witer period with their family---paired with 'Circus Circus' that has 3 stars, a gym, and free internet, but no other amenities---will obtain a Score of:

In [ ]:
model_sm.predict([[1,4,2,3,.75,3,1,0,0,0,1,0,0,0,1,0,0,0,1]])